In [78]:
import tensorflow as tf

## Follow the official "get started" 

References:

- https://www.tensorflow.org/get_started/get_started

In [33]:
# create scalar constant tensors
# scalars are rank 0 tensor

node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0, dtype=tf.float32)
print(node1)
print(node2)

# shape is empty because they are scalars
print(node1.get_shape())
print(node2.get_shape())


Tensor("Const_2:0", shape=(), dtype=float32)
Tensor("Const_3:0", shape=(), dtype=float32)
()
()


In [43]:
# tensors must be evaluated within a session
# Session.run() takes tensors and evaluate them
with tf.Session() as sess:
    print(sess.run([node1, node2]))
    
# tf.rank() returns the rank of an object as a tensor
# rank zero means a scalar
with tf.Session() as sess:
    print(sess.run([tf.rank(node1), tf.rank(node2)])) 

[3.0, 4.0]
[0, 0]


In [44]:
# tensors support a variety of operations
# tf.add(x, y) creates a tensor that adds up two tensors x and y
node3 = tf.add(node1, node2)
print(node3)
with tf.Session() as sess:
    print(sess.run(node3))

Tensor("Add_15:0", shape=(), dtype=float32)
7.0


In [42]:
# placeholders defines a tensor for which values are supplied later 
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = tf.add(a, b)
with tf.Session() as sess:
    print(sess.run(c, {a: -1.0, b: 3}))

# vector values can be fed
with tf.Session() as sess:
    print(sess.run(c, {a: [-1.0, 10.0], b: [3.0, 4.0]}))

2.0
[  2.  14.]


In [ ]:
# Variables are trainable parameters
W = tf.Variable([0.3], dtype=tf.float32, name="weight")
b = tf.Variable([-0.3], dtype=tf.float32, name="bias")
x = tf.placeholder(tf.float32)
yhat = tf.add(tf.multiply(W, x), b, name="yhat")

print(W)
print(b)
print(x)
print(yhat)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(yhat, {x: [1, 2, 3, 4]}))
    

In [62]:
# compare and train with true values
y = tf.placeholder(tf.float32)
loss = tf.reduce_mean(tf.square(tf.subtract(y, yhat)))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(loss, {x: [1, 2, 3, 4], y: [0.1, -1.1, -1.8, -3.0]}))

# set values mannually to W and b to improve the fit
# `tf.assign()` defines an operation to change variable values
# note that this does not happen until given to `run`
fixW = tf.assign(W, [-1.0])
fixb = tf.assign(b, [1.0])
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run([fixW, fixb])
    print(sess.run(loss, {x: [1, 2, 3, 4], y: [0.1, -1.1, -1.8, -3.0]}))


5.735
0.015


In [77]:
# supervised training to find better variables values
# `train` here defines an operation to reduces the loss by
# the gradient descent
# again, this shall be given to `run` to invoke it
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # update the variables for multiple times
    for i in range(1000):
        sess.run(train, {x: [1, 2, 3, 4], y: [0.1, -1.1, -1.8, -3.0]})
    
    print(sess.run(loss, {x: [1, 2, 3, 4], y: [0.1, -1.1, -1.8, -3.0]}))
    print(sess.run([W, b]))
    # or print the values by
    print(W.eval(), b.eval())

0.0135716
[array([-0.97275525], dtype=float32), array([ 0.96989703], dtype=float32)]
[-0.97275525] [ 0.96989703]


## Computation graph example

References: 

- Saito, K (2016). Deep Learning from Scratch, O'Reilly Japan.

In [107]:
# Fig 5-17. p.140
num_apple    = tf.placeholder(tf.float32)
price_apple  = tf.Variable([100.0], tf.float32)
num_orange   = tf.placeholder(tf.float32)
price_orange = tf.Variable([150.0], tf.float32)
taxrate      = tf.Variable([0.1], tf.float32)

pretax       = tf.add(tf.multiply(num_apple,  price_apple),
                      tf.multiply(num_orange, price_orange))
expenditure  = tf.add(pretax, tf.multiply(pretax, taxrate))
print(num_apple)
print(price_apple)
print(num_orange)
print(price_orange)
print(taxrate)
print(pretax)
print(expenditure)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(expenditure, {num_apple: 2, num_orange: 3}))

Tensor("Placeholder_53:0", dtype=float32)
<tf.Variable 'Variable_55:0' shape=(1,) dtype=float32_ref>
Tensor("Placeholder_54:0", dtype=float32)
<tf.Variable 'Variable_56:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'Variable_57:0' shape=(1,) dtype=float32_ref>
Tensor("Add_41:0", dtype=float32)
Tensor("Add_42:0", dtype=float32)
[ 715.]


In [108]:
# compute the grandients
# tf.gradients(y, x) to get dy/dx

grads = tf.gradients(
    expenditure, 
    [num_apple, price_apple, num_orange, price_orange, taxrate]
)
print(grads)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(grads, {num_apple: 2, num_orange: 3}))

[<tf.Tensor 'gradients_23/Mul_48_grad/Reshape:0' shape=<unknown> dtype=float32>, <tf.Tensor 'gradients_23/Mul_48_grad/Reshape_1:0' shape=(1,) dtype=float32>, <tf.Tensor 'gradients_23/Mul_49_grad/Reshape:0' shape=<unknown> dtype=float32>, <tf.Tensor 'gradients_23/Mul_49_grad/Reshape_1:0' shape=(1,) dtype=float32>, <tf.Tensor 'gradients_23/Mul_50_grad/Reshape_1:0' shape=(1,) dtype=float32>]
[110.0, array([ 2.20000005], dtype=float32), 165.0, array([ 3.30000019], dtype=float32), array([ 650.], dtype=float32)]


## Minimizing quadratic functions

In [123]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = tf.placeholder(tf.float32)
x = tf.Variable(0.0, tf.float32)

y = a*x*x + b*x + c

print(a)
print(b)
print(c)
print(x)
print(y)

# find the mimimum
optimizer = tf.train.GradientDescentOptimizer(0.1)
train = optimizer.minimize(y)

data = {a: 0.5, b: 5.0, c: 3.0}
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # update the variables for multiple times
    for i in range(100):
        sess.run(train, data)
    
    print(sess.run([x, y], data))

# expected: x = -b/(2a)
x_star = -data[b]/data[a]/2
y_star = data[a]*x_star*x_star + data[b]*x_star + data[c]
print("expected: x = ", x_star, "y = ", y_star)

Tensor("Placeholder_97:0", dtype=float32)
Tensor("Placeholder_98:0", dtype=float32)
Tensor("Placeholder_99:0", dtype=float32)
<tf.Variable 'Variable_72:0' shape=() dtype=float32_ref>
Tensor("add_21:0", dtype=float32)
[-4.999867, -9.499999]
expected: x =  -5.0 y =  -9.5
